##Install libraries

In [ ]:
# Install the necessary packages
!pip install --upgrade transformers
!pip install transformers
!pip install datasets
!pip install torch
!pip install sentencepiece # Needed for some tokenizers
!pip install numpy pandas
!pip install streamlit
!pip install localtunnel
!nvidia-smi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 54.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are

##Import Libraries

In [ ]:
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import random
import streamlit as st

##Load a pretrained Language Model

In [ ]:
# Load GPT-2 lightweight tokenizer and model
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Ensure use of GPU if available for faster processing
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Model loaded successfully!")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded successfully!


##Set up a basic chatbot function

In [ ]:
# Simple chatbot function
def chatbot_response(prompt, max_length=50):
  inputs = tokenizer(prompt, return_tensors="pt").to(device)
  outputs = model.generate(
      **inputs,
      max_length = max_length,
      pad_token_id = tokenizer.eos_token_id,
      do_sample = True,
      top_k = 50,
      top_p = 0.95
  )

  response = tokenizer.decode(outputs[0], skip_special_tokens = True)
  return response

# Example usage
print("Chatbot is ready! Type 'quit' to exit.")
while True:
  user_input = input("You:")
  if user_input.lower() == 'quit':
    break
  response = chatbot_response(user_input)
  print(f"Chatbot: {response}")

Chatbot is ready! Type 'quit' to exit.


##Preprocessing

In [5]:
# Load EmpatheticDialogues dataset
print("Loading dataset...")
dataset = load_dataset("empathetic_dialogues", split = "train")

# Preprocess the dataset to create user-chatbot pairs
def preprocess(example):
  return {"text": f"User: {example['context']}\nChatbot: {example['utterance']}"}

# Apply preprocesseing
train_dataset = dataset.map(preprocess, remove_columns = dataset.column_names)
print("Preprocessed example:")
print(train_dataset[0]) # Check the format of the preprocessed dataset

Loading dataset...


README.md:   0%|          | 0.00/7.15k [00:00<?, ?B/s]

empathetic_dialogues.py:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

The repository for empathetic_dialogues contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/empathetic_dialogues.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/76673 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10943 [00:00<?, ? examples/s]

Map:   0%|          | 0/76673 [00:00<?, ? examples/s]

Preprocessed example:
{'text': 'User: sentimental\nChatbot: I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.'}


##Tokenize the dataset

In [6]:
# Load the GPT-2 tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Add padding token to GPT-2 tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(example):
    # Tokenize the text and duplicate it for 'labels'
    inputs = tokenizer(example['text'], truncation=True, padding="max_length", max_length=128)
    inputs['labels'] = inputs['input_ids'].copy()  # Add labels for loss calculation
    return inputs

# Apply tokenization
tokenized_dataset = train_dataset.map(tokenize_function, batched=True)

# Check the format of the tokenized dataset
print(tokenized_dataset[0])


Map:   0%|          | 0/76673 [00:00<?, ? examples/s]

{'text': 'User: sentimental\nChatbot: I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.', 'input_ids': [12982, 25, 46908, 198, 30820, 13645, 25, 314, 3505, 1016, 284, 766, 262, 26056, 351, 616, 1266, 1545, 13, 632, 373, 262, 717, 640, 356, 1683, 3377, 640, 3436, 1978, 13, 4900, 612, 373, 257, 1256, 286, 661, 62, 785, 2611, 62, 356, 2936, 588, 262, 691, 661, 287, 262, 995, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 5

##Fine-Tune GPT-2

In [ ]:
# Disable Weights & Biases logging
import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM

# Load the GPT-2 model and tokenizer
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Tokenized dataset example (ensure you already have tokenized_dataset defined)
# Replace this line with your tokenized dataset preparation if not defined earlier
# tokenized_dataset = ...

# Training arguments with checkpoint saving logic
training_args = TrainingArguments(
    output_dir = "./results",  # Output directory
    evaluation_strategy = "steps", # Evaluate periodically
    eval_steps = 1000, # Evaluate every 1000 steps
    logging_steps = 500, # Log every 500 steps
    learning_rate = 3e-5,  # Learning rate
    per_device_train_batch_size = 8,  # Adjust batch size if needed
    per_device_eval_batch_size = 8,
    num_train_epochs = 3,  # Number of epochs
    weight_decay = 0.01,
    save_total_limit = 2,  # Save only the latest two checkpoints
    save_steps = 2000,  # Save checkpoint every 500 steps
    logging_dir="./logs",  # Log directory
    save_strategy="steps"  # Save checkpoints at specified steps
)

# Split the dataset into train and validation
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Convert tokenized dataset to Hugging Face Dataset if needed
if not isinstance(tokenized_dataset, Dataset):
    tokenized_dataset = Dataset.from_dict(tokenized_dataset)

# Split the dataset into train and validation using Hugging Face's train_test_split
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

# Assign splits to variables
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Train data
    eval_dataset=eval_dataset     # Eval data
)

# Fine-tune the model
print("Starting fine-tuning...")
trainer.train()
print("Fine-tuning completed!")

# Save the final fine-tuned model and tokenizer
final_model_dir = "./fine_tuned_gpt2"
model.save_pretrained(final_model_dir)
tokenizer.save_pretrained(final_model_dir)
print(f"Final model and tokenizer saved to {final_model_dir}")


##Test the Fine-Tuned Model

In [ ]:
# Reload the fine-tuned model
from transformers import pipelines

fine_tuned_model = "./fine_tuned_gpt2"
chatbot = pipeline("text-generation", model = fine_tuned_model, tokenizer = fine_tuned_model)

#Test the chatbot
prompt = "I'm feeling really down and alone"
response = chatbot(prompt, max_length = 50, do_sample = True, top_k = 50, top_p = 0.95)
print("Chatbot:", response[0]['generated_text'])

##Save Colab Progress

In [ ]:
model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")
print("Model and tokenizer saved!")

##Streamlit Build

In [ ]:
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# ------------------------------
# Title and Introduction Section
st.markdown("""
    <style>
    /* Global app background */
    .stApp {
        background: linear-gradient(to bottom, #FBFAFC, #E0E5EC);
    }

    /* Welcome Container */
    .welcome-container {
        background: linear-gradient(to bottom, #FBFAFC, #E0E5EC);
        border-radius: 16px;
        padding: 20px;
        margin: 20px auto;
        box-shadow: 8px 8px 16px #d1d9e6, -8px -8px 16px #ffffff;
        text-align: center;
    }
    .welcome-container h1 {
        font-family: 'Arial', sans-serif;
        font-size: 28px;
        color: #333333;
    }
    .welcome-container p {
        font-family: 'Verdana', sans-serif;
        font-size: 16px;
        color: #555555;
    }

    /* Neumorphic Button */
    .neumorphic-button {
        display: inline-block;
        background: linear-gradient(145deg, #E6E8EC, #FFFFFF);
        border: none;
        border-radius: 12px;
        box-shadow: 8px 8px 15px #d1d9e6, -8px -8px 15px #ffffff;
        padding: 10px 20px;
        font-size: 16px;
        font-weight: bold;
        color: #333333;
        cursor: pointer;
        text-align: center;
        margin: 10px;
    }
    .neumorphic-button:active {
        box-shadow: inset 8px 8px 15px #d1d9e6, inset -8px -8px 15px #ffffff;
    }

    /* Chat Area Styling */
    .chat-area {
        background-color: #FBFAFC;
        border-radius: 16px;
        padding: 15px;
        box-shadow: inset 8px 8px 16px #d1d9e6, inset -8px -8px 16px #ffffff;
        font-family: 'Verdana', sans-serif;
        font-size: 14px;
        color: #333333;
    }
    </style>
""", unsafe_allow_html=True)

st.markdown("""
<div class="welcome-container">
    <h1>Welcome to EmpathAI</h1>
    <p>A chatbot fine-tuned on emotional support data to assist you during difficult moments.<br>
    Start chatting below to share your thoughts or feelings.</p>
</div>
""", unsafe_allow_html=True)

# ------------------------------
# Load Model and Tokenizer
@st.cache(allow_output_mutation=True)
def load_model():
    model_name = "fine_tuned_gpt2"  # Replace with fine-tuned model directory
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return tokenizer, model

tokenizer, model = load_model()

# ------------------------------
# Chat interface
st.write("---")
st.write("**Chat with the Bot Below:**")

# Input Area
user_input = st.text_input("You:", key="user_input")

# Generate Chat Response
if user_input:
    with st.spinner("Thinking..."):
        # Tokenize input
        inputs = tokenizer(user_input, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
        # Generate response
        outputs = model.generate(**inputs, max_length=150, pad_token_id=tokenizer.eos_token_id)
        # Decode response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        st.markdown(f'<div class="chat-area"><strong>Chatbot:</strong> {response}</div>', unsafe_allow_html=True)

# ------------------------------
# Clear Chat Button
if st.button("Clear Chat", key="clear"):
    st.experimental_rerun()


In [ ]:
!streamlit run chatbot_app.py & npx localtunnel --port 8501